In [1]:
# import dependencies
import pandas as pd

In [2]:
# define file path and read in file
heroes_file = 'Resources/HeroesOfPymoli_purchase_data.csv'
heroes_df = pd.read_csv(heroes_file)

In [3]:
# ensure file read in properly
heroes_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
# ensure no missing or faulty data 
heroes_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [5]:
# display the total number of players
player_count = len(heroes_df['SN'].unique())
tot_players_df = pd.DataFrame({"Total Players": [player_count]})
tot_players_df

,Total Players
0,576


In [6]:
# Puchasing Analysis (Total)
num_of_unique_items = len(heroes_df['Item ID'].unique())
avg_price = heroes_df['Price'].mean()
num_of_purchases = len(heroes_df['Purchase ID'].unique())
tot_revenue = heroes_df['Price'].sum()
tot_analysis = [{'Number of Unique Items': num_of_unique_items,
                  'Average Price': avg_price,
                  'Number of Purchases': num_of_purchases,
                  'Total Revenue': tot_revenue}]
tot_analysis_df = pd.DataFrame(tot_analysis)
tot_analysis_df.style.format({'Average Price': '${:.2f}', 'Total Revenue': '${:,.2f}'})

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [7]:
# Gender Demographics
gender_df = heroes_df.drop_duplicates(['SN'])
gender_count = gender_df['Gender'].value_counts()
gender_count_df = pd.DataFrame(gender_count)
gender_count_df = gender_count_df.rename(columns={'Gender':'Total Count'})
gender_count_df['Percentage of Players'] = gender_count/player_count
gender_count_df.style.format({'Percentage of Players': '{:.2%}'})

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [8]:
# Purchasing Analysis (Gender)
purchase_count = heroes_df['Gender'].value_counts()
tot_purchase_value = heroes_df.groupby('Gender').Price.sum()
avg_purchase_price = tot_purchase_value/purchase_count
avg_total_pp = tot_purchase_value/gender_count
purchase_summary_gender_df = pd.DataFrame({'Purchase Count': purchase_count,
                                    'Average Purchase Price': avg_purchase_price,
                                          'Total Purchase Value': tot_purchase_value,
                                          'Avg Total Purchase per Person': avg_total_pp})
purchase_summary_gender_df.style.format({'Average Purchase Price': '${:.2f}',
                                        'Total Purchase Value': '${:,.2f}',
                                        'Avg Total Purchase per Person': '${:.2f}'})

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [9]:
# Age Demographics
age_df = gender_df.copy()
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 100]
age_bin_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
age_df['Age Range'] = pd.cut(gender_df['Age'], age_bins, labels=age_bin_names, include_lowest=True)
age_count = age_df['Age Range'].value_counts()
pct_of_players = age_count/player_count
age_summary_df = pd.DataFrame({'Total Count': age_count, 'Percentage of Players': pct_of_players})
age_summary_df.sort_index(axis = 0).style.format({'Percentage of Players': '{:.2%}'})

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [10]:
# Purchase Analysis (Age)
purchase_df = heroes_df.copy()
purchase_df['Age Ranges'] = pd.cut(purchase_df['Age'], age_bins, labels=age_bin_names, include_lowest=True)
pur_count = purchase_df['Age Ranges'].value_counts()
tot_pur_value = purchase_df.groupby('Age Ranges').Price.sum()
avg_pur_price = tot_pur_value/pur_count
avg_tot_pp = tot_pur_value/age_count
purchase_summary_age_df = pd.DataFrame({'Purchase Count': pur_count,
                                        'Average Purchase Price': avg_pur_price,
                                        'Total Purchase Value': tot_pur_value,
                                        'Avg Total Purchase per Person': avg_tot_pp})
purchase_summary_age_df.style.format({'Average Purchase Price': '${:.2f}',
                                        'Total Purchase Value': '${:,.2f}',
                                        'Avg Total Purchase per Person': '${:.2f}'})

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [11]:
# Top Spenders
spenders_df = heroes_df.copy()
spenders_pur_count = spenders_df['SN'].value_counts()
spenders_tot_purchase = spenders_df.groupby('SN').Price.sum()
spenders_avg_pur_price = spenders_tot_purchase/spenders_pur_count
spenders_summary_df = pd.DataFrame({'Purchase Count': spenders_pur_count,
                                   'Average Purchase Price': spenders_avg_pur_price,
                                   'Total Purchase Value': spenders_tot_purchase})
spenders_summary_df = spenders_summary_df.sort_values('Total Purchase Value', ascending=False)
spenders_summary_df.head().style.format({'Average Purchase Price': '${:.2f}',
                                        'Total Purchase Value': '${:,.2f}'})

,Purchase Count,Average Purchase Price,Total Purchase Value
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [12]:
# Most Popular Items
popular_df = heroes_df[['Item ID','Item Name','Price']]
def purchase_count(group):
    pc = group['Item Name'].count()
    group['Purchase Count'] = pc
    return group
def total_purchase(group):
    tp = group['Price'].sum()
    group['Total Purchase Value'] = tp
    return group
popular_df = popular_df.groupby(['Item ID','Item Name']).apply(purchase_count)
popular_df = popular_df.groupby(['Item ID','Item Name']).apply(total_purchase)
popular_df = popular_df.drop_duplicates(['Item Name'])
popular_df = popular_df.sort_values('Purchase Count', ascending=False)
popular_df.set_index(['Item ID','Item Name'], inplace = True, drop = True)
popular_df.rename(columns = {'Price':'Item Price'}, inplace = True)
popular_df = popular_df[['Purchase Count', 'Item Price', 'Total Purchase Value']]
popular_df.head().style.format({'Item Price': '${:.2f}',
                                        'Total Purchase Value': '${:,.2f}'})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22


In [13]:
# Most Profitable Items
profitable_df = popular_df.sort_values('Total Purchase Value', ascending=False)
profitable_df.head().style.format({'Item Price': '${:.2f}',
                                        'Total Purchase Value': '${:,.2f}'})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.88,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
